# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle as pck
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import The Dataset

In [ ]:
train = pd.read_csv('/kaggle/input/pe-files-malwares/dataset_malwares.csv', sep=',')
test = pd.read_csv('/kaggle/input/pe-files-malwares/dataset_test.csv', sep=',')

In [ ]:
train

In [ ]:
test

In [ ]:
train.isnull().any().sum()

# Split The Train Data


In [ ]:
# #The target is Malware Column {0=Benign, 1=Malware}

X = train.drop(['Name', 'Malware'], axis = 1)
y = train['Malware']

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=101)
X_train.head()

In [ ]:
# Feature Scaling

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

In [ ]:
X_new = pd.DataFrame(X_scaled, columns = X.columns)

In [ ]:
X_new.head()

In [ ]:
# Principal Component Analysis

skpca = PCA(n_components = 55)
X_pca = skpca.fit_transform(X_new)
print('Variance sum : ', skpca.explained_variance_ratio_.cumsum()[-1])

In [ ]:
# Build The Model
model = RandomForestClassifier(n_estimators = 100, random_state = 0, oob_score = True, max_depth = 16, max_features = 'sqrt')
model.fit(X_pca, y_train)

In [ ]:
X_test_scaled = scaler.transform(X_test)
X_new_test = pd.DataFrame(X_test_scaled, columns = X.columns)
X_test_pca = skpca.transform(X_new_test)

In [ ]:
# Predict

y_pred = model.predict(X_test_pca)

In [ ]:
print(classification_report(y_pred, y_test))

In [ ]:
sns.heatmap(confusion_matrix(y_pred, y_test), annot = True, fmt = 'd', cmap = plt.cm.Blues, cbar = False)

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('scale', scaler),('pca', skpca),('clf', model)])

In [ ]:
X_testing = test.drop('Name', axis = 1)

X_testing_scaled = pipe.named_steps['scale'].transform(X_testing)
X_testing_pca = pipe.named_steps['pca'].transform(X_testing_scaled)
y_testing_pred = pipe.named_steps['clf'].predict_proba(X_testing_pca)

In [ ]:
pd.concat([test['Name'], pd.DataFrame(y_testing_pred)], axis = 1)